# Convert follow to graph¶

https://docs.rapids.ai/api/cugraph/stable/api.html#cugraph.structure.convert_matrix.from_cudf_edgelist

cnx.structure.convert_matrix.from_cudf_edgelist

cnx.community.subgraph_extraction.subgraph

cnx.community.spectral_clustering.???

cnx.community.triangle_count.triangles

cnx.community.kernighan_lin.kernighan_lin_bisection


In [62]:
#!echo Available GPUs: $CUDA_VISIBLE_DEVICES

In [2]:
import os

import pandas as pd
pd.options.mode.chained_assignment = None
import networkx as nx

### CUDA RAPIDS
import cugraph as cnx
import cudf
import cupy as cp

path = '../proj/data/'

In [3]:
follow = pd.read_csv((path + 'Following_Ignoring_Relationships_01052019_31052019.csv'), sep=';')
t1 = follow[follow.ID_CommunityConnectionType == 1]#.iloc[:,:2] #
t2 = follow[follow.ID_CommunityConnectionType == 2]#.iloc[:,:2] #
t2.loc[:,'ID_CommunityConnectionType'] = t2.loc[:,'ID_CommunityConnectionType'].replace(2, -1)

Fg = nx.DiGraph()
Fg.add_nodes_from(t1.iloc[:,0].unique())
Fg.add_nodes_from(t1.iloc[:,1].unique())
Fg.add_nodes_from(t2.iloc[:,0].unique())
Fg.add_nodes_from(t2.iloc[:,1].unique())

e1 = list(zip(t1.ID_CommunityIdentity, t1.ID_CommunityIdentityConnectedTo))
e2 = list(zip(t2.ID_CommunityIdentity, t2.ID_CommunityIdentityConnectedTo))

### Weighted > t1.ID_CommunityConnectionType
Fg.add_edges_from(e1, w=1)
Fg.add_edges_from(e2, w=-1)
print(len(Fg.nodes()), len(Fg.edges()) )

17485 86691


In [ ]:
ignore = []
for _, (s,t) in enumerate(zip(t2.ID_CommunityIdentity, t2.ID_CommunityIdentityConnectedTo)):
    if (s in A3di.keys()) & (t in A3di.keys()):
        #print(s,t)
        ignore.append((s,t))
    #print(_)
len(ignore)

## Between centrality

In [11]:
#between = cnx.centrality.betweenness_centrality(Fg)

## Louvain

In [4]:

## convert to undirected for some algos
udFg = Fg.to_undirected()

louvain = cnx.community.louvain(udFg)
print(louvain[1], '= modularity_score') 
len(set(louvain[0].values()))

0.41152438521385193 = modularity_score


1130

## spectral Balanced Cut Clustering

In [79]:
specC = cnx.spectralBalancedCutClustering(udFg, 5)
#specC.get(1)tis

In [50]:
#specC

In [56]:
#cnx.community.spectral_clustering.analyzeClustering_edge_cut(udFg, 5, specC) ##, 'vertex', 'cluster')
## specC should be DF with cols

In [4]:
#[ e for e in Fg.edges_iter () ]

## resistance_distance

In [77]:
#nx.algorithms.distance_measures.resistance_distance(Fg)

## katz centrality
top-20

In [76]:
sorted(cnx.centrality.katz_centrality(Fg).items(), key=lambda x: x[1], reverse=True)[0:20]

[(524430, 0.13207079261472443),
 (529517, 0.1240546014552086),
 (167449, 0.10872699367352559),
 (29262, 0.10535807206007657),
 (157302, 0.1033338073561847),
 (538574, 0.08821299051965677),
 (578557, 0.08787622322220892),
 (309385, 0.08761107270187267),
 (654383, 0.08704175254055774),
 (207112, 0.08595183583519314),
 (606270, 0.08490994350853526),
 (687729, 0.08282162298098292),
 (536203, 0.08260217491449354),
 (130491, 0.08083588430091924),
 (551403, 0.08065022650631884),
 (527635, 0.08056980733433014),
 (170447, 0.08047408398695834),
 (579004, 0.08035697187826843),
 (655677, 0.07897687425775635),
 (519605, 0.07888398980327685)]